In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from utils.constants.dir_path import DataDirPath
from utils.constants.constants import Constants
from utils.convert_csv import ConvertCsv

In [70]:
base_info = DataDirPath.base_structure_info()
dir_path = '/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data'
dirs = glob.glob(f'{dir_path}/*')

In [71]:
PATH = Constants.path()
base_df = pd.read_csv(f'{base_info}/base_info.csv')
base_df

,structure,structure_idx,natom,E,Vol
0,alpha-critobalite,1,48,-1011.449548,839.055430
1,alpha-critobalite,2,48,-1011.449396,839.055430
2,alpha-critobalite,3,48,-1011.449194,839.055430
3,alpha-critobalite,4,48,-1011.448339,839.055430
4,alpha-critobalite,5,48,-1011.448402,839.055430
...,...,...,...,...,...
8545,stishovite,696,48,-1010.442985,331.977432
8546,stishovite,697,48,-1010.386112,331.977432
8547,stishovite,698,48,-1010.322835,331.977432
8548,stishovite,699,48,-1010.398489,331.977432


In [72]:
# 各構造ごとのstructure_idxを全構造のindexに変更する
plus = 0
for structure in PATH:
    structure = structure.split('_')[-1]
    base_df.loc[base_df.structure == structure, ['structure_idx']] += plus
    idx = base_df[base_df.structure == structure].structure_idx.unique()
    idx.sort()
    plus = idx[-1]
    print(structure,idx[0],idx[-1])

alpha-critobalite 1 1150
alpha-quartz 1151 2250
beta-quartz 2251 3050
beta-trydymite 3051 4300
coesite 4301 5500
Fdd2-beta-critobalite 5501 6850
hex-trydymite 6851 7850
stishovite 7851 8550


In [73]:
dirs.sort()
with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(['type','structure','r-cut','num-pairs','MAE','MSE','RMSE','R2'])
for directory in dirs:
    files = glob.glob(f'{directory}/*')

    test_file = glob.glob(f'{directory}/test*')[0].split('/')[-1]
    train_file = glob.glob(f'{directory}/train*')[0].split('/')[-1]

    obj = ConvertCsv(directory,directory)
    obj.convert_n2p2_output_to_csv(10,test_file,'tmp_test')
    obj.convert_n2p2_output_to_csv(10,train_file,'tmp_train')

    df_test = pd.read_csv(f'{directory}/tmp_test.csv')
    df_train = pd.read_csv(f'{directory}/tmp_train.csv')
    # テストに対して
    df_test_new = pd.merge(df_test,base_df,left_on='index',right_on='structure_idx')

    mae_test = mean_absolute_error(df_test_new['Eref'],df_test_new['Ennp'])
    mse_test = mean_squared_error(df_test_new['Eref'],df_test_new['Ennp'])
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(df_test_new['Eref'],df_test_new['Ennp'])

    r_cut = directory.split('/')[-1].split('_')[1]
    num_pairs = directory.split('/')[-1].split('_')[2]

    with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(['test','all',r_cut,num_pairs,mae_test,mse_test,rmse_test,r2_test])
    # 訓練データに対して
    df_train_new = pd.merge(df_train,base_df,left_on='index',right_on='structure_idx')

    mae_train = mean_absolute_error(df_train_new['Eref'],df_train_new['Ennp'])
    mse_train = mean_squared_error(df_train_new['Eref'],df_train_new['Ennp'])
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(df_train_new['Eref'],df_train_new['Ennp'])

    r_cut = directory.split('/')[-1].split('_')[1]
    num_pairs = directory.split('/')[-1].split('_')[2]

    with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(['train','all',r_cut,num_pairs,mae_train,mse_train,rmse_train,r2_train])


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [75]:
directory = dirs[9]
files = glob.glob(f'{directory}/*')

test_file = glob.glob(f'{directory}/test*')[0].split('/')[-1]
train_file = glob.glob(f'{directory}/train*')[0].split('/')[-1]

obj = ConvertCsv(directory,directory)
obj.convert_n2p2_output_to_csv(10,test_file,'tmp_test')
obj.convert_n2p2_output_to_csv(10,train_file,'tmp_train')

df_test = pd.read_csv(f'{directory}/tmp_test.csv')
df_train = pd.read_csv(f'{directory}/tmp_train.csv')
# テストに対して
df_test_new = pd.merge(df_test,base_df,left_on='index',right_on='structure_idx')
df_test_new
#
# mae_test = mean_absolute_error(df_test_new['Eref'],df_test_new['Ennp'])
# mse_test = mean_squared_error(df_test_new['Eref'],df_test_new['Ennp'])
# rmse_test = np.sqrt(mse_test)
# r2_test = r2_score(df_test_new['Eref'],df_test_new['Ennp'])

# r_cut = directory.split('/')[-1].split('_')[1]
# num_pairs = directory.split('/')[-1].split('_')[2]
#
# with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
#     writer = csv.writer(f)
#     writer.writerow(['test','all',r_cut,num_pairs,mae_test,mse_test,rmse_test,r2_test])


,index,Eref,Ennp,structure,structure_idx,natom,E,Vol
0,33,0.164652,-NAN,alpha-critobalite,33,48,-1010.992930,839.055430
1,358,-1.005827,-NAN,alpha-critobalite,358,48,-1011.373891,522.736660
2,386,0.027873,-NAN,alpha-critobalite,386,48,-1010.649379,522.736660
3,531,0.096495,-NAN,alpha-critobalite,531,48,-1010.909912,908.013213
4,1045,0.545922,-NAN,alpha-critobalite,1045,48,-1010.448241,814.934288
...,...,...,...,...,...,...,...,...
824,7936,0.157981,-NAN,stishovite,7936,48,-1010.923925,350.302260
825,8258,-1.178945,-NAN,stishovite,8258,48,-1011.452768,345.233252
826,8414,-0.991375,-NAN,stishovite,8414,48,-1011.371762,408.644169
827,8454,-1.199050,-NAN,stishovite,8454,48,-1011.460060,395.003095
